# Model training with YOLOv5

## Prepare YOLO

This step is pretty easy, we only have to download YOLOv5 from Ultralytics and install the necessary dependencies

### Download YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5

### We must slightly edit the requirements to have a container-compatible version of openCV

In [ ]:
!sed -i 's/opencv-python/opencv-python-headless/' yolov5/requirements.txt

### Then install the requirements

In [ ]:
!pip install -qr yolov5/requirements.txt

### Unfortunately, some subdependencies may have messed up with OpenCV. So let's make sure again we have the right version again...

In [ ]:
!pip uninstall -qy opencv-python
!pip uninstall -qy opencv-python-headless
!pip install -q opencv-python-headless

## Training

### Create the configuration file

There is a `configuration.yaml` file already present in the folder. Verify that it has the right number and names of the classes you want to use and that you downloaded.

### Freeze the YOLOv5 Backbone
The backbone means the layers that extract input image features. We will freeze the backbone so the weights in the backbone layers will not change during YOLOv5 transfer learning. We will then only train the last layers (the head layers).

In this example, we will use the smallest available base model, yolov5n. If we open the file `yolov5/models/yolov5n.yaml` we can see the following structure:

In the **backbone** section we can see that there are 10 layers. So that's the number of layers we want to freeze in our training!

### Some training parameters (hyperparameters) we are going to use

#### The base model
weights = yolov5n.pt

This is the smallest available model, to save processing time in this example. Adjust for better results.
Other models are available here: https://github.com/ultralytics/yolov5#pretrained-checkpoints

####  The number of training iterations
epochs = 50

This is voluntarily low to save processing time for this example. Adjust for better results (>150).

#### The number of images analyzed in a single pass
batch = 256

You may have to adjust this depending on the memory available on your GPU. The higher (in a power of 2) the better, until you run out of memory...

#### Number of layers to freeze
freeze = 10

As per above, we want to freeze 10 layers

### We can now launch the training!

NOTE: PyTorch first caches images to speed up the process. If you have enough memory and shared memory that is not an issue. However, this may not be always the case, especially with large datasets. Therefore the cache is forced to disk here. But of course this is a parameter you can change.

In [ ]:
!python yolov5/train.py --data configuration.yaml --weights yolov5n.pt --epochs 50 --batch 256 --freeze 10 --cache disk

The model has been saved as `yolov5n.pt`.

### Test the model

We can now simply test the model against an image: `test/img_test.jpg`
The image will be analyzed, enriched with bounding boxes, and the result will be saved under `yolov5/runs/detect/exp`

In [ ]:
!python yolov5/detect.py --weights yolov5n.pt --img 640 --conf-thres 0.2 --source test/img_test.jpg

![Result](./yolov5/runs/detect/exp/img_test.jpg)